<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

100% 2.54G/2.55G [02:01<00:00, 23.0MB/s]
100% 2.55G/2.55G [02:01<00:00, 22.5MB/s]
--2020-12-09 11:59:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-09 11:59:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-09 11:59:50--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 20

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from preprocessrecipeqa import *
import random

In [4]:
batch_size = 1
(train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)
l_embed = get_embedding_layer(word_index, embedding_index)


In [8]:
for i, label in train_iter1:
  for j in i[0]: print(j)
  # print(i[0]['recipe_id'])
  for k in i[0]['context']:
    print(k['body'])
    print(len(k['images']))
  break

recipe_id
context
choice_list
question
[20192, 11837, 27206, 17641, 2590, 14579, 9782, 795, 12409, 11827, 10972, 19896, 3682, 23563, 19190, 28977, 6235, 21435, 4462, 1263, 14216, 3671, 9880, 14490, 18706, 795, 10972, 12571, 8229, 11827, 27206, 5989, 3452, 16444, 5191, 12984, 8921, 27044, 21594, 27032, 679, 27206, 11456, 4244, 6844, 16127, 11837, 9726, 12571, 18775, 16967, 17425, 11827, 12409, 3080, 10176, 20105, 27831, 27206, 17950, 15189, 10176, 24089, 27831, 9234, 12571, 2995, 25664, 21594, 16967, 24929, 9880, 7162, 1086, 2718, 10972, 12583, 21594, 27206, 7400, 11827, 22195, 11456, 7318, 12571, 17170, 16967, 3176, 10972, 25025, 16581, 28369, 1086, 27206, 16461, 9880, 4934, 2583, 21435, 10176, 855, 19342, 6844, 27206, 9172, 16967, 28139, 14799, 1486, 27233, 18584, 21594, 5913, 10176, 855, 5382, 19342, 18706, 16967, 8837, 27206, 7986, 23686, 27032, 17425, 3682, 28139, 11991, 19337, 21594, 17452, 21594, 25327, 16967, 28369, 7101, 3682, 12370, 9880, 3583, 9880, 2069, 13174, 23340, 795, 2

In [13]:
def define_model():
  img_inputs = keras.Input(shape=(224, 224, 3))
  l1 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l1(img_inputs)
  l2 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l2(x)
  l3 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l3(x)
  l4 = keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same')
  x = l4(x)
  flatten = keras.layers.Flatten()
  x = flatten(x)
  densex = keras.layers.Dense(100,activation='tanh')
  x = densex(x)
  text_inputs = keras.Input(shape = (None,100))
  lstm = tf.keras.layers.LSTM(100)

  densey =tf.keras.layers.Dense(100,activation='tanh')
  y = densey(lstm(text_inputs))
  
  concat = keras.layers.Concatenate()
  fc1 = keras.layers.Dense(500,activation = 'relu')
  z = fc1(concat([x,y]))
  fc2 = keras.layers.Dense(500,activation = 'relu')
  z = fc2(z)
  fc3 = keras.layers.Dense(2,activation = 'softmax')
  outputs = fc3(z)
  model = tf.keras.Model(inputs=[text_inputs,img_inputs], outputs=outputs)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
  return model

In [14]:
model = define_model()

In [15]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 112, 112, 64) 4864        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 56, 56, 64)   102464      conv2d_8[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 28, 28, 64)   102464      conv2d_9[0][0]                   
_______________________________________________________________________________________

In [16]:
model.predict([l_embed(np.ones([1,10])),np.ones([1,224,224,3])])

array([[0.52147526, 0.47852474]], dtype=float32)

3.361645576660521e-05

In [21]:
for i, label in train_iter1:
  for batch in i: 
    if len(batch['context'])==1:
      continue
  # print(i[0]['recipe_id'])
    for k in range(len(batch['context'])):
      # print(k['body'])
      loss = 0
      context = batch['context'][k]
      if len(context['images'])==0:
        continue
      input_text = l_embed(tf.tile([context['body']],[len(context['images']),1]))
      input_images = tf.convert_to_tensor (context['images'])
      labels = np.ones([len(input_text)])
      if input_text.shape[1]!=0:
        loss= model.train_on_batch([input_text,input_images],labels)
        # print(model.predict([input_text,input_images]))
      
      j = random.randint(0,len(batch['context'])-1)
      while j==k:
        j = random.randint(0,len(batch['context'])-1)
      fake_context = batch['context'][j]
      input_text2 = l_embed(tf.tile([fake_context['body']],[len(context['images']),1]))
      labels2 = np.zeros([len(input_text)])
      if input_text2.shape[1]!=0:
        loss+=model.train_on_batch([input_text2,input_images],labels2)
        loss/=2
      print(loss)


0.7309737205505371
0.6982189118862152
0.7037436068058014
0.7099738717079163
0.7297621369361877
0.7126275599002838
0.7170794606208801
0.7091774642467499
0.7046544551849365
0.7082193791866302
0.7040002644062042
0.6899984180927277
0.6935310661792755
0.7080830931663513
0.7070668935775757
0.6980837285518646
0.6991521120071411
0.6999296844005585
0.7040022313594818
0.7035335600376129
0.6997336149215698
0.7037566304206848
0.6932750940322876
0.7081494927406311
0.7187564969062805
0.6985742151737213


KeyboardInterrupt: ignored

In [20]:
t=0
a=0
for i, label in val_iter1:
  for batch in i: 
  # print(i[0]['recipe_id'])
    for k in range(len(batch['context'])):
      loss = 0
      context = batch['context'][k]
      if len(context['images'])==0:
        continue
      input_text = l_embed(tf.tile([context['body']],[len(context['images']),1]))
      input_images = tf.convert_to_tensor (context['images'])
      if input_text.shape[1]!=0:
        logits = (model.predict([input_text,input_images]))
        for i in logits:
          t+=1
          if i[1]>0.5:
            a+=1
      # j = random.randint(0,len(batch['context'])-1)
      # while j==k:
      #   j = random.randint(0,len(batch['context'])-1)
      # fake_context = batch['context'][j]
      # input_text2 = l_embed(tf.tile([fake_context['body']],[len(context['images']),1]))
      # if input_text2.shape[1]!=0:
      #   logits=model.predict([input_text2,input_images])
      #   for i in logits:
      #     t+=1
      #     if i[1]<0.5:
      #       a+=1
  print(a/t)
        # print(model.predict([input_text,input_images]))


0.10526315789473684
0.2
0.20689655172413793
0.2
0.21176470588235294
0.23333333333333334
0.22448979591836735
0.2018348623853211
0.19469026548672566
0.18253968253968253
0.2
0.20134228187919462
0.20253164556962025
0.21176470588235294
0.21264367816091953
0.21978021978021978
0.21428571428571427
0.2107843137254902
0.215311004784689
0.2145922746781116
0.22709163346613545


KeyboardInterrupt: ignored